<h1><b>ĐỒ ÁN CUỐI KỲ NMKHDL</b></h1>
<h2><b>POKEDEX</b></h2>

Họ tên: Nguyễn Đăng Quang

MSSV: 18120527



# Giới thiệu sơ lược
##### Có thể bạn chưa biết, Pocket Monsters viết tắt là Pokemon, một series game cũng như anime đình đám, series game Pokemon phát triển bởi GameFreak hiện vẫn đang làm mưa làm gió trên các hệ máy Nitendo và trên thị trường game toàn cầu. Nhân dịp gen9 (Scarlet & Violet) của game vừa ra mắt, chúng ta sẽ điểm lại các pokemon từ những thế hệ trước và thông số cũng như đặc tính của chúng thông qua Pokedex. Ở đây, chúng ta sẽ sử dụng "https://pokemondb.net/pokedex/all" là trang web chứa dữ liệu Pokedex, hay nói cách khác là danh sách của toàn bộ Pokemon ĐÃ xuất hiện xuyên suốt series game này. Let's go!

# Install và import
- bs4 (Beautiful Soup): Một thư viện Python dùng để phân tích các tài liệu HTML và XML. Nó tạo ra một cây phân tích cú pháp cho các trang được phân tích cú pháp có thể được sử dụng để trích xuất dữ liệu từ HTML, rất hữu ích cho web scrapping.
- requests: Cho phép gửi các yêu cầu HTTP bằng Python.

In [ ]:
!pip install bs4
!pip install requests

In [ ]:
import bs4
import requests
import pandas as pd

# Thu thập và xử lý dữ liệu
### 1. Lấy danh sách các id riêng biệt của từng Pokemon
- Lưu ý: Có thể các Pokemon có rất nhiều dạng (vùng miền, mega evolution, mega X/mega Y, nguyên thủy (primal form), tấn công/phòng thủ,...), hình dạng Pokemon thay đổi nhưng id thì vẫn giữ nguyên, vì vậy sẽ xuất hiện trường hợp một id lặp lại nhiều lần, chúng ta chỉ xét hình dạng cơ bản của chúng nên chỉ cần lấy 1 id cho mỗi Pokemon thôi. 

In [ ]:
url = 'https://pokemondb.net/pokedex/all'

source = requests.get(url).text
soup = bs4.BeautifulSoup(source,'html.parser')

# Thu thập id của toàn bộ pokemon
pokemon_id = soup.findAll('span', {'class': 'infocard-cell-data'})

id = []

for i in range(len(pokemon_id)):
    id.append(pokemon_id[i].text)
    id[i] = int(id[i])

id = list(set(id))      # Sắp xếp và xóa các id bị trùng

### 2. Danh sách các url của từng Pokemon
- Chúng ta có thể dùng id hoặc tên của Pokemon để làm subdirectory của url tương ứng với Pokemon đó. 

##### Ví dụ: Pokemon có tên là Bulbasaur, id là 001:

https://pokemondb.net/pokedex/bulbasaur

https://pokemondb.net/pokedex/001 hay https://pokemondb.net/pokedex/1

In [ ]:
# Lấy danh sách urls cho web scrapping
urls = []
n = len(id)

for i in range(n):
    urls.append('https://pokemondb.net/pokedex/' + str(id[i]))

### 3. Thu thập dữ liệu
#### 3.1. Thông tin của một Pokemon
- Lưu ý: ID của Pokemon sẽ được thêm vào sau cùng sau đó đặt ID làm cột index, ở đây chúng ta chỉ xét các thuộc tính như: tên, hệ, loài,...
- Sẽ có thêm các cơ sở dữ liệu riêng để mô tả chi tiết các đặc tính cũng như đặc tính ẩn và tương khắc hệ
- Các thuộc tính của một Pokemon sẽ bao gồm:

            - name: Tên

            - types_list: Hệ của Pokemon, mỗi con sẽ mang 1 hệ hoặc song hệ

            - species: Loài

            - height: Chiều cao

            - weight: Cân nặng

            - ability: Đặc tính

            - hid_ability (hidden ability): Đặc tính ẩn

            - catch_rate: Tỉ lệ bắt trúng bằng Poke Ball, bị ảnh hưởng bởi rất nhiều yếu tố:
                    * HP hiện tại
                    * Loại bóng (Poke Ball, Great Ball, Ultra Ball,...), Master Ball có tỷ lệ thu phục là 100% với tất cả Pokemon
                    * Trạng thái hiện tại (Choáng, phỏng, nhiễm độc, ngủ, tê liệt,...)
        
            - base_fs (base friendship): Chỉ số thân thiện cơ bản, một số Pokemon có chỉ số thân thiện là 0, bọn này có vẻ không ưa con người cho lắm

            - base_exp: Số lượng exp cần thiết để lên 1 level bất kỳ được tính theo hàm mũ. Mỗi loại có công thức khác nhau và khá phức tạp:
                    * Ví dụ 1 công thức đơn giản nhất cho mọi người:
                        Loại 1 000 000: exp = n^3 
                        n là level cần lên.
                        -> Để lên lv3 Pokemon loại này cần tổng cộng là 27exp trừ số exp hiện có sẽ ra số exp cần phải luyện thêm
            
            - growth_rate: Tốc độ tăng trưởng:
                    * Erratic: 600.000 exp ở level 100
                    * Fast: 800.000 điểm kinh nghiệm ở cấp 100
                    * Medium Fast: 1.000.000 điểm kinh nghiệm ở cấp 100
                    * Medium Slow: 1.059.860 điểm kinh nghiệm ở cấp 100
                    * Slow: 1.250.000 điểm kinh nghiệm ở cấp 100
                    * Fluctuating: 1.640.000 exp ở cấp 100:
        
            - genders: Giới tính theo tỷ lệ

            - Base Stats: Chỉ số cơ bản, bao gồm: 
                    * hp (Health Point): Sinh mệnh
                    * atk (Attack): Tấn công
                    * defn (Defense): Phòng thủ, giáp (Vì def là khai báo hàm nên không dùng để khai báo biến cho Defense được :v)
                    * sp_atk (Special Attack): Tấn công đặc biệt, sát thương phép
                    * sp_def (Special Defense): Phòng thủ đặc biệt, kháng phép
                    * spd (Speed): Tốc độ (tương tự như các game xoay tua, Pokemon tốc độ cao hơn sẽ được đánh trước)
                    
            - total: Tổng base stats


- Hàm get_pokemon_info(soup) giúp lấy ra thông tin của một Pokemon có ID chỉ định. Sau đó sẽ dùng vòng lặp để lấy thông tin toàn bộ Pokemon trong Pokedex

In [ ]:
# Thu thập dữ liệu nào
def get_pokemon_info(soup):
    # name
    name = soup.find('h1').text

    # types
    types_list = soup.find_all('td')[1].text.replace('\n', '')
    types_list = types_list[:-1].replace(' ', ' & ')        # type1 & type2

    # species
    species = soup.find_all('td')[2].text

    # height
    height = soup.find_all('td')[3].text.replace('\xa0', '')

    # weight
    weight = soup.find_all('td')[4].text.replace('\xa0', '')

    # bility & hidden Ability
    ability = soup.find_all('span', {'class': 'text-muted'})[0].text.replace('1. ', '')
    hid_ability = soup.find_all('small', {'class': 'text-muted'})[0].text.replace(' (hidden ability)', '')

    # catch_rate
    catch_rate = soup.find_all('td')[8].text.replace('\n', '')

    # base friendship 
    base_fs = soup.find_all('td')[9].text.replace('\n', '')

    # base exp
    base_exp = soup.find_all('td')[10].text.replace('\n', '')

    # growth_rate
    growth_rate = soup.find_all('td')[11].text.replace('\n', '')

    # genders
    genders = soup.find_all('td')[13].text.replace('\n', '')

    # base stats (base, min, max): health point, attack, defense, special attack, special defense, speed, total (base)
    hp = int(soup.find_all('td')[15].text.replace('\n', ''))

    atk = int(soup.find_all('td')[19].text.replace('\n', ''))

    defn = int(soup.find_all('td')[23].text.replace('\n', ''))

    sp_atk = int(soup.find_all('td')[27].text.replace('\n', ''))
    
    sp_def = int(soup.find_all('td')[31].text.replace('\n', ''))

    spd =  int(soup.find_all('td')[35].text.replace('\n', ''))

    total = int(soup.find_all('td')[39].text)

    return name, types_list, species, height, weight, ability, hid_ability, catch_rate, base_fs, base_exp, growth_rate, genders, hp, atk, defn, sp_atk, sp_def, spd, total

#### 3.2. Thông tin của toàn bộ Pokemon trong Pokedex
- Ta sẽ lấy danh sách Pokemon bằng cách truy cập vào từng url để lấy thông tin từng con, việc này sẽ tốn khá nhiều thời gian

In [ ]:
data = []
for url in urls:
    source = requests.get(url).text
    soup = bs4.BeautifulSoup(source,'html.parser')
    data.append(get_pokemon_info(soup))


- Sau khi đã có danh sách thông tin của toàn bộ Pokemon trong Pokedex, khởi tạo một cơ sở dữ liệu để lưu trữ
- Index ở đây sẽ là ID của từng Pokemon, bắt đầu từ 1.

In [50]:
pokedex = pd.DataFrame(data = data, columns=['Name', 'Type', 'Species', 'Height', 'Weight', 'Ability', 'Hidden Ability',
                                             'Catch Rate', 'Base Friendship', 'Base Exp', 'Growth Rate', 'Gender',
                                             'HP', 'Attack', 'Defense', 'Sp. Atk', 'Sp. Def', 'Speed', 'Total'], index = id)

,Name,Type,Species,Height,Weight,Ability,Hidden Ability,Catch Rate,Base Friendship,Base Exp,Growth Rate,Gender,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,Total
1,Bulbasaur,Grass & Poison,Seed Pokémon,0.7m (2′04″),6.9kg (15.2lbs),Overgrow,Chlorophyll,"45 (5.9% with PokéBall, full HP)",50 (normal),64,Medium Slow,"87.5% male, 12.5% female",45,49,49,65,65,45,318
2,Ivysaur,Grass & Poison,Seed Pokémon,1.0m (3′03″),13.0kg (28.7lbs),Overgrow,Chlorophyll,"45 (5.9% with PokéBall, full HP)",50 (normal),142,Medium Slow,"87.5% male, 12.5% female",60,62,63,80,80,60,405
3,Venusaur,Grass & Poison,Seed Pokémon,2.0m (6′07″),100.0kg (220.5lbs),Overgrow,Chlorophyll,"45 (5.9% with PokéBall, full HP)",50 (normal),236,Medium Slow,"87.5% male, 12.5% female",80,82,83,100,100,80,525
4,Charmander,Fire,Lizard Pokémon,0.6m (2′00″),8.5kg (18.7lbs),Blaze,Solar Power,"45 (5.9% with PokéBall, full HP)",50 (normal),62,Medium Slow,"87.5% male, 12.5% female",39,52,43,60,50,65,309
5,Charmeleon,Fire,Flame Pokémon,1.1m (3′07″),19.0kg (41.9lbs),Blaze,Solar Power,"45 (5.9% with PokéBall, full HP)",50 (normal),142,Medium Slow,"87.5% male, 12.5% female",58,64,58,80,65,80,405
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
901,Ursaluna,Ground & Normal,Peat Pokémon,2.4m (7′10″),290.0kg (639.3lbs),Guts,Unnerve,"20 (2.6% with PokéBall, full HP)",50 (normal),—,—,"50% male, 50% female",130,140,105,45,80,50,550
902,Basculegion,Water & Ghost,Big Fish Pokémon,3.0m (9′10″),110.0kg (242.5lbs),Rattled,Mold Breaker,"45 (5.9% with PokéBall, full HP)",50 (normal),—,—,"100% male, 0% female",120,112,65,80,75,78,530
903,Sneasler,Poison & Fighting,Free Climb Pokémon,1.3m (4′03″),43.0kg (94.8lbs),Pressure,Poison Touch,"20 (2.6% with PokéBall, full HP)",50 (normal),—,—,"50% male, 50% female",80,130,60,40,80,120,510
904,Overqwil,Dark & Poison,Pin Cluster Pokémon,2.5m (8′02″),60.5kg (133.4lbs),Poison Point,Intimidate,"45 (5.9% with PokéBall, full HP)",50 (normal),—,—,"50% male, 50% female",85,115,95,65,65,85,510


- Load dữ liệu vào file 'pokemon_db.csv'

In [43]:
pokedex.to_csv('pokemon_db.csv', encoding='utf-8', index = True)